In [ ]:
# authorize access to the drive
# This allows me to access files (Data) in my google drive
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
# This downloads Hugging Face Transformers, tools we need to fine-tune GPT-2
# need an older version of transformers
!pip3 install transformers==3.3.1
#!pip3 install git+https://github.com/huggingface/transformers@3.3.1
#!pip3 install git+https://github.com/huggingface/transformers

     |████████████████████████████████| 1.1MB 23.2MB/s 
     |████████████████████████████████| 1.1MB 54.6MB/s 
     |████████████████████████████████| 3.0MB 52.6MB/s 
  Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Found existing installation: transformers 4.0.0rc1
    Uninstalling transformers-4.0.0rc1:
      Successfully uninstalled transformers-4.0.0rc1


In [ ]:
# Allows me to go to the correct directory in my google drie
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

[Errno 2] No such file or directory: 'drive/My Drive/School Work/Fourth Year/Capstone Everything'
/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [ ]:
# the fine-tuning code by Hugging Face that allows us to fine-tune
# this file is already archieved after the most recent hugging face modelling se we need to replace the url
# !wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/contrib/legacy/run_language_modeling.py

--2020-11-29 03:41:47--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/contrib/legacy/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13684 (13K) [text/plain]
Saving to: ‘run_language_modeling.py.1’

run_language_modeli 100%[===================>]  13.36K  --.-KB/s    in 0.001s  

2020-11-29 03:41:47 (10.6 MB/s) - ‘run_language_modeling.py.1’ saved [13684/13684]



In [ ]:
% ls

'Brian'\''s Peer Feedback Template (10.28.21).gdoc'
'Capstone Coversheet.gdoc'
'Capstone Proposal (Junior Year).gdoc'
 class/
'Dev Journal.gdoc'
 Experiments/
 Models/
 new_run_language_modeling.py
 None/
'Paper Drafts'/
'Punch Card.gsheet'
 run_language_modeling.py
 run_language_modeling.py.1
 runs/


In [ ]:
# get dataset
datapath = "Models/Data"
train_path = datapath + "/train.txt"
valid_path = datapath + "/valid.txt"
output = "./output"

In [ ]:
# %%bash 
# export DIRECTORY=Models/Data/Description_Data
# ls $DIRECTORY

In [ ]:
# Main fine-tuning code using run_language_modeling.py
%%bash
export TRAIN_FILE=Models/Data/train.txt
export TEST_FILE=Models/Data/valid.txt
export MODEL_NAME=gpt2
export OUTPUT_DIR=Models/Experiments/Claims/3epochs

# python3 run_language_modeling.py \
#     --output_dir=$OUTPUT_DIR \
#     --model_type=$MODEL_NAME \
#     --model_name_or_path=$MODEL_NAME \
#     --do_train \
#     --train_data_file=$TRAIN_FILE \
#     --do_eval \
#     --eval_data_file=$TEST_FILE \
#     --per_gpu_train_batch_size=1 \
#     --save_steps=-1 \
#     --num_train_epochs=2

python run_language_modeling.py \
    --output_dir=$OUTPUT_DIR \
    --model_type=gpt2 \
    --model_name_or_path=$MODEL_NAME \
    --do_train \
    --train_data_file=$TRAIN_FILE \
    --do_eval \
    --eval_data_file=$TEST_FILE \
    --per_gpu_train_batch_size=1 \
    --save_steps=-1 \
    --num_train_epochs=3

{'loss': 1.84433251953125, 'learning_rate': 3.495788206979543e-05, 'epoch': 0.9025270758122743, 'total_flos': 382279090176000, 'step': 500}
{'loss': 1.52551025390625, 'learning_rate': 1.9915764139590855e-05, 'epoch': 1.8050541516245486, 'total_flos': 764558180352000, 'step': 1000}
{'loss': 1.38601025390625, 'learning_rate': 4.873646209386281e-06, 'epoch': 2.707581227436823, 'total_flos': 1146837270528000, 'step': 1500}
{'eval_loss': 1.3252715138915163, 'epoch': 3.0, 'total_flos': 1270695695745024, 'step': 1662}


2020-11-29 03:41:59.061329: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/transformers/training_args.py:332: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/29/2020 03:42:00 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/29/2020 03:42:00 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='Models/Experiments/Claims/3epochs', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=1, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-0

In [ ]:
# this allow us to generate sample patent claims 

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import numpy as np

OUTPUT_DIR = "./Models/Experiments/Claims/3epochs"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
# we use the GPT-2 model to load our fine-tuned model and get the tokenization
tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)
                                        
def generate(input_str, length=250, n=5):
  '''
  This is the main generation code using our model
  '''
  cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
  model.eval()
  with torch.no_grad():
    for i in range(length):
      outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0,-1], dim=0)
      next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
      cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    return output_text

def choose_from_top(probs, n=5):
    '''
    This chooses the top n choices and probabilistically select one choice
    '''
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)
# print the generated text
generated_text = generate("This aparatus \n", n=5)
print(generated_text)

This aparatus 
<BOS> 1.  An organic light-emitting device comprising a pixel array and a pixel array comprising: an organic layer disposed in the image sensor layer on the pixel array and including a first organic layer and a second organic layer on the organic layer, wherein the pixel array comprises: a first electrode;  a first organic layer disposed in the first electrode;  a first pixel;  a second organic layer disposed in the second electrode;  a second organic layer disposed in the second electrode;  a second pixel;  a pixel of the pixel array;  a pixel array comprising the first organic layer and the second organic layer;  and at least one pixel of the pixel array includes a first pixel and a first pixel offset, wherein each of the first pixel and the second pixel are independently pixel pixels. <EOS>
<BOS> 6.  The method of claim 4, further comprising: performing one or more chemical vapor deposition operations in the organic light emitting diode array. <EOS>
<BOS> 6.  The disp

In [ ]:
% ls output

ls: cannot access 'output': No such file or directory
